In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset, Subset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import torch.optim as optim

In [2]:
import torch_directml
dml = torch_directml.device()
print(dml)

privateuseone:0


In [3]:
# Load the dataset with data augmentation and normalizing
class Cifar10(Dataset):
    def __init__(self):
        super().__init__()
        transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.RandomAutocontrast(),
            transforms.ToTensor(),
            transforms.Resize((128, 128)), # B2 for resizing image
            transforms.Normalize(mean=[0.4854, 0.4567, 0.4062], std=[0.2291, 0.2249, 0.2253])
        ])
        self.data = ImageFolder("clean_train", transform=transform_train)

    def __len__(self):
        return self.data.__len__()

    def __getitem__(self, idx):
        features, label = self.data[idx]
        return features, label

    def split_data(self):
        indices = np.arange(len(self.data))
        targets = np.array(self.data.targets)
        train_indices, val_indices = train_test_split(indices,
                                                       test_size=0.2,
                                                       random_state=42,
                                                       stratify=targets)
        final_train_set = Subset(self.data, train_indices)
        final_val_set = Subset(self.data, val_indices)
        return final_train_set, final_val_set
        
train_data = Cifar10()

In [4]:
# Split the dataset
train_set, val_set = train_data.split_data()

In [5]:
# Build the CNN
import torch.nn as nn

class CnnCifar(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), # B4. Padding in each CONV layer
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), # B4. Padding in each CONV layer
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
        )
        self.classifier = nn.Linear(128 * 32 * 32, num_classes)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x


In [6]:
# Feed training data into the model
import torch
batch_size = 128
train_load = DataLoader(train_set, shuffle=True,  batch_size=batch_size,
                        num_workers=4, pin_memory=True, persistent_workers=True)
val_load   = DataLoader(val_set,   shuffle=False, batch_size=batch_size,
                        num_workers=4, pin_memory=True, persistent_workers=True)
cnn = CnnCifar().to(dml)
criterion = nn.CrossEntropyLoss().to(dml)
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

for epoch in range(10):
    run_amount = 0
    for images, labels in train_load:
        images = images.to(memory_format=torch.channels_last).to(dml, non_blocking=True)
        labels = labels.to(dml, non_blocking=True)
        
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        run_amount += loss.item()
    avg_loss = run_amount / len(train_load)
    print(f"Epoch [{epoch+1}/10], Loss: {avg_loss:.4}")

C:\Users\ancho\anaconda3\envs\tf_env\Lib\site-packages\torch\nn\functional.py:1593: UserWarning: The operator 'aten::elu.out' is not currently supported on the DML backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at C:\__w\1\s\pytorch-directml-plugin\torch_directml\csrc\dml\dml_cpu_fallback.cpp:17.)
  result = torch._C._nn.elu(input, alpha)


Epoch [1/10], Loss: 1.846
Epoch [2/10], Loss: 1.5
Epoch [3/10], Loss: 1.431
Epoch [4/10], Loss: 1.382
Epoch [5/10], Loss: 1.346
Epoch [6/10], Loss: 1.316
Epoch [7/10], Loss: 1.289
Epoch [8/10], Loss: 1.271
Epoch [9/10], Loss: 1.262
Epoch [10/10], Loss: 1.257


In [7]:
# Evaluate